In [1]:
"""
批量读取.csv特征集(bicps or tricps)
对每个人的数据进行单独归一化再merge
"""
import pandas as pd

from utils_iFEMG_feature import df_save_csv, df_norm, df_merge_colname, df_norm_clinical

In [ ]:
# healthy curl 0-2kg 完整数据集名单
'''
0912s1, 1017s2: 0-1kg
0926s2 只包含了2kg和伸展0.5kg最大值
'''
['w8s1', '0912s2', '0919s2', '0926s1', '1010s1', '1017s1', '1024s1', '1107s1']

In [ ]:
# healthy extension 0-1kg 完整数据集名单
'''
只有0kg: 0912s1(患侧<0), 0912s2, 0926s1(患侧<0), 
0 - 0.5: 0926s2(健侧只有0.5), '1010s1',
'''
['0919s2']

In [2]:
"""
获取文件路径下多个文件夹中.csv格式的特征集

1. 文件夹路径
2. 肌肉bi tri
"""
# 文件夹路径 健康人数据
folder_path = r"E:\Data\paper2\积水潭患者数据集"
subject_name_list = ['w8s1', '0912s1', '0912s2', '0919s2', '0926s1', '0926s2', '1010s1', '1017s1', '1017s2', '1024s1', '1107s1']


file_df_list = []
for name in subject_name_list:
    file_path_healthy = folder_path + '\\' + name + '\\healthy\\' + name + "_curl_abs.csv"
    file_path_affected = folder_path + '\\' + name + '\\affected\\' + name + "_curl_abs.csv"
    try:
        temp_df_healthy = pd.read_csv(file_path_healthy, header = [0,1], index_col = None)
        temp_df_affected = pd.read_csv(file_path_affected, header = [0,1], index_col = None)
        temp_df_healthy[('subject_info', 'side')] = 'healthy'
        temp_df_affected[('subject_info', 'side')] = 'affected'
        temp_df = pd.concat([temp_df_healthy, temp_df_affected], axis=0, ignore_index=True)
        
    except FileNotFoundError:
        print(f"文件 {name} 未读取")
    
    file_df_list.append(temp_df)
    pass

print(f"文件数: {len(file_df_list)}")


文件数: 11


In [3]:
"""
对每名被试信号特征分别单独归一化

NOTE:
    - 需要修改保存文件的文件名！！
"""

# 归一化某特征
normed_df_list = [] 
err_df_index = 0    # 如果有key error发生报错，报告所在df的index
# 需要进行归一化的特征列名
col_name2sacle = [("bicps_br", "initial_pressure_ave"),
                    ("bicps_br", "FMG"),
                    ("bicps_br", "mav"),
                    ("bicps_br", "rms"),
                    ("bicps_br", "wave_length"),
                    ("bicps_br", "zero_crossing"),
                    ("bicps_br", "slope_sign_change"),
                    ("bicps_br", "mean_freq"),
                    ("bicps_br", "mean_power_freq"),
                    ("tricps_br_medial", "initial_pressure_ave"),
                    ("tricps_br_medial", "FMG"),
                    ("tricps_br_medial", "mav"),
                    ("tricps_br_medial", "rms"),
                    ("tricps_br_medial", "wave_length"),
                    ("tricps_br_medial", "zero_crossing"),
                    ("tricps_br_medial", "slope_sign_change"),
                    ("tricps_br_medial", "mean_freq"),
                    ("tricps_br_medial", "mean_power_freq"),
                    ("tricps_br_lateral", "initial_pressure_ave"),
                    ("tricps_br_lateral", "FMG"),
                    ("tricps_br_lateral", "mav"),
                    ("tricps_br_lateral", "rms"),
                    ("tricps_br_lateral", "wave_length"),
                    ("tricps_br_lateral", "zero_crossing"),
                    ("tricps_br_lateral", "slope_sign_change"),
                    ("tricps_br_lateral", "mean_freq"),
                    ("tricps_br_lateral", "mean_power_freq")]

for df in file_df_list:
    # 对指定的列进行归一化
    try:
        normed_df_list.append(df_norm_clinical(dataframe=df,
                                               col_name=col_name2sacle,
                                               method='z_score',
                                               train_col_name=('subject_info', 'label'),
                                               train_col_values=[0, 0.5, 1]))
    except KeyError:
        print(f"index {err_df_index} 存在错误")
        pass
    err_df_index = err_df_index + 1
    pass
# 合并dataframe
final_data_df = pd.concat(normed_df_list, axis = 0, ignore_index = True)

In [4]:
# 合并列名
final_data_df_n = df_merge_colname(final_data_df)

final_data_df_n

,subject_info_subject_name,subject_info_height,subject_info_weight,subject_info_age,subject_info_gender,subject_info_label,subject_info_side,bicps_br_initial_pressure_ave,bicps_br_FMG,bicps_br_mav,...,tricps_br_medial_mean_power_freq,tricps_br_lateral_initial_pressure_ave,tricps_br_lateral_FMG,tricps_br_lateral_mav,tricps_br_lateral_rms,tricps_br_lateral_wave_length,tricps_br_lateral_zero_crossing,tricps_br_lateral_slope_sign_change,tricps_br_lateral_mean_freq,tricps_br_lateral_mean_power_freq
0,w8s1,163,77.5,73,0,0.0,healthy,0.264332,-0.834709,1.333790,...,-0.920293,0.580450,0.376447,-0.302707,-0.257013,-0.174564,0.597931,0.935593,-0.404117,-0.021018
1,w8s1,163,77.5,73,0,0.0,healthy,0.264332,-1.060881,0.589367,...,0.967061,0.580450,0.401577,-0.906481,-0.666368,-0.888070,0.558823,1.108320,0.510007,0.712348
2,w8s1,163,77.5,73,0,0.0,healthy,0.264332,-0.873370,0.607650,...,1.132278,0.580450,0.442973,-0.768296,-0.565227,-0.874164,0.188775,0.862873,-0.153251,0.382974
3,w8s1,163,77.5,73,0,0.0,healthy,0.264332,-0.960192,0.330866,...,1.015542,0.580450,0.456464,-1.054028,-0.770760,-1.156951,0.427841,1.028323,-0.259272,0.145059
4,w8s1,163,77.5,73,0,0.0,healthy,0.264332,-0.961252,0.330415,...,1.150688,0.580450,0.484242,-1.120775,-0.831344,-1.210279,0.629919,1.226824,0.031978,0.488862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,1107s1,150,60.0,66,0,2.0,affected,2.071787,1.432518,1.158777,...,0.963333,-0.160464,-0.101044,0.033244,-0.086020,0.248078,-1.023727,-0.976712,-1.027272,-0.925162
641,1107s1,150,60.0,66,0,2.0,affected,2.071787,1.482564,0.907694,...,1.081050,-0.160464,0.388158,0.336959,0.141355,0.343284,-1.128817,-1.212494,-1.083809,-1.122362
642,1107s1,150,60.0,66,0,2.0,affected,2.071787,1.542811,1.680064,...,0.860542,-0.160464,0.481561,0.299037,0.158678,0.389070,-1.035922,-1.148199,-1.116932,-1.125568
643,1107s1,150,60.0,66,0,2.0,affected,2.071787,1.525316,1.274841,...,0.918776,-0.160464,0.065509,0.321341,0.115970,0.423688,-1.115121,-1.074502,-1.082077,-1.081129


In [5]:
# 保存文件
df_save_csv(final_data_df_n, r"E:\Data\paper2\积水潭患者数据集\iFEMG_curl_abs_normed_all.csv")

File E:\Data\paper2\积水潭患者数据集\iFEMG_curl_abs_normed_all.csv saved!
